# Finding the positive roots of $f(x)=x^2+\cos(30e\cdot x)$

In [46]:
import time
import math
import numpy as np
from numpy import random 

def f(x):        # Function whose roots are to be found
    return x**2+math.cos(30*math.e*x)

def Df(x):         # Derivative of the function
    return 2*x-30*math.e*math.sin(30*math.e*x)


def sgn(x):      # The signum function
    if(x>0):
        return 1   # If x>0, return 1
    elif(x<0):
        return -1    # If x<0, return -1
    else:
        return 0 # If x=0, return 0

step=2*math.pi/(2*30*math.e)    # Step size for the subintervals


def newtonraphson(initial_approx):   #Newton Raphson method
    x=initial_approx
    if(Df(x)==0):
        return None
    #Newton Raphson method
    x_new=x-f(x)/Df(x)  # Newton Raphson formula
    while(abs(f(x_new))>10**(-6) or (abs(x_new-x)/abs(x_new))>10**(-6)): # Negation of the statement (P and Q)
                                                            # is equivalent to (not P) or (not Q)
        x=x_new
        x_new=x-f(x)/Df(x)
        if(x_new<initial_approx-step/2 or x_new>initial_approx+step/2):   # Checking if the approximation is in the interval, 
                        #if not, return None
            return None
        if(f(x_new)==0):
            return x_new
    return x_new



def mullermethod(x0,x1,x2,epsilon):  # Muller's method
    init_left=x0
    init_right=x2
    init_mid=x1
    epsilon=10**(-6)
    while(abs(x1-x2)/abs(x2)>epsilon or abs(f(x2))>epsilon):
        
        a=((x1-x2)*(f(x0)-f(x2))-(x0-x2)*(f(x1)-f(x2)))/((x0-x2)*(x1-x2)*(x0-x1))    # Coefficients of the quadratic polynomial
        b=((x0-x2)**2*(f(x1)-f(x2))-(x1-x2)**2*(f(x0)-f(x2)))/((x0-x2)*(x1-x2)*(x0-x1)) 
        c=f(x2)
        if(b**2-4*a*c<0): #check if the discriminant is negative, if yes, return None
            return None
        x3=x2-2*c/(b+sgn(b)*math.sqrt(b**2-4*a*c)) # Muller's formula

        x0,x1,x2=x1,x2,x3
        if(x0<init_left or x2>init_right):  # Checking if the approximation is in the interval,                 
            return None     #if not, stop the iteration and return None
    return x3




def secantmethod(init_left,init_right):   # Secant method
    a=init_left
    b=init_right
    if(f(b)==f(a)):     # Checking if the denominator is zero, if yes, return None
        return None
    c=(a*f(b)-b*f(a))/(f(b)-f(a))
    while(abs(f(c))>10**(-6) or abs(c-b)/abs(c)>10**(-6)):
        a=b
        b=c
        if(f(b)==f(a)):  #Denominator check
            return None
        c=(a*f(b)-b*f(a))/(f(b)-f(a))
        if(f(c)==0):
            return c
        if(a<init_left or b>init_right):    # Checking if the approximation is in the interval, if not, return None
            return None
    return c
        


def regulafalsi(init_left,init_right):    # Regula Falsi method
    a=init_left
    b=init_right
    if(f(b)==f(a)):         #Checking if the denominator is zero, if yes, return None
        return None
    c=(a*f(b)-b*f(a))/(f(b)-f(a))
    old=b
    current=c
    while(abs(f(current))>10**(-6) or abs(current-old)/abs(old)>10**(-6)):
        old=c
        if(f(b)==f(a)):    #Denominator check
            #Although regula falsi always converges, it's better to place a check in case
            #the difference is rounded off to 0 due to floating points
            return None
        c=(a*f(b)-b*f(a))/(f(b)-f(a))
        if(f(c)==0):
            return c
        if(f(a)*f(c)<0):
            b=c
            current=b
        else:
            a=c
            current=a
    return current 
    


def bisection(init_left,init_right):  # Bisection method
                                      # Bisection method is guaranteed to converge, so no need to return None at any stage
    a=init_left
    b=init_right
    old=b
    c=(a+b)/2
    current=c
    while(abs(f(current))>10**(-6) or abs(current-old)/abs(current)>10**(-6)):
        old=current
        c=(a+b)/2
        if(f(c)*f(a)<0):
            b=c
            current=b
        else:
            if(f(c)==0):
                return c
            else:
                a=c
                current=a
    return current

#tracking_list is a list of tuples, where each tuple is of the form (method_name,root)

def root_finder(x,y,tracking_list):    # Function to find the root in the interval by the fastest possible method
                         # The order of convergence of the methods is Newton Raphson, Muller's Method, 
                         # Secant Method, Regula Falsi, Bisection Method
    newtonraphson_output=newtonraphson((x+random.uniform(0,step)))   # Random initial approximation for Newton Raphson in [x,x+step]
    if(newtonraphson_output!=None):  #If None is returned, Newton Raphson has failed, otherwise return the root
        tracking_list+=[("Root found by Newton Raphson",newtonraphson_output)]
        return newtonraphson_output
    

    mullermethod_output=mullermethod(x,x+step/2,x+step,10**(-6))   #If Newton Raphson fails,                                                     
                                                                #try Muller's Method, with endpoints as initial approximations
    if(mullermethod_output!=None):
        tracking_list+=[("Root found by Muller's Method",mullermethod_output)]
        return mullermethod_output
    
    secantmethod_output=secantmethod(x,y)    #If Newton Raphson fails, try Secant Method, with endpoints as initial approximations
    if(secantmethod_output!=None):
        tracking_list+=[("Root found by Secant Method",secantmethod_output)]
        return secantmethod_output
    

    regulafalsi_output=regulafalsi(x,y)      #If Secant Method fails, try Regula Falsi, with endpoints as initial approximations
    if(regulafalsi_output!=None):
        tracking_list+=[("Root found by Regula Falsi",regulafalsi_output)]
        return regulafalsi_output
    

    bisection_output=bisection(x,y)          #If Regula Falsi fails, try Bisection Method, with endpoints as initial approximations
                                             #Bisection Method is guaranteed to converge, so no need to check for None
    tracking_list+=[("Root found by Bisection Method",bisection_output)]
    
    return bisection_output




start=time.time()     # Start time of the program
root_list=list()      # Initial blank List of roots
endpoints=[]          # List of endpoints of the subintervals, where the function changes sign.

for i in np.arange(0,1+step,step):   #np.arange(a,b,step) divides the interval [a,b] into subintervals of length "step", starting from a
                                     #but it does not include b, so we have to add an extra step to include b
    if(f(i)*f(i+step)<0):     # Checking if the function changes sign in the interval
        endpoints+=[i]         # If yes, add the endpoint to the list of endpoints
    if(f(i)==0):              # Checking if the function is zero at the endpoint, if yes, add it to the list of roots
        root_list+=[i]

tracking_list=list()    # List of tuples, where each tuple is of the form (method_name,root)
for i in range(len(endpoints)):
    root=root_finder(endpoints[i],endpoints[i]+step,tracking_list)      # Finding the root in the interval using the root_finder function
    root_list+=[root]

end=time.time()      # End time of the program

print("Roots are: ",root_list)       
print("Number of roots: ",len(root_list))

print("Rounded off to 6 decimal places: ",[round(i,6) for i in root_list])
print("Time taken: ",end-start," seconds")

for i in tracking_list:
    print(i)

Roots are:  [0.01926667444973937, 0.057745477121806055, 0.09642462865853345, 0.13461263923147074, 0.17372926819273105, 0.21133548241535155, 0.2511817852020291, 0.2879141628004701, 0.32878425885546086, 0.36434765115466805, 0.4065406272521091, 0.4406327178252044, 0.48445820242186005, 0.5167624229346484, 0.562550102008478, 0.5927237260656256, 0.6408395285480515, 0.6684932685699284, 0.7193683834581519, 0.7440288265725794, 0.7982180285305519, 0.8192486113888032, 0.8775750852356653, 0.8939655213594536, 0.9581044457083825, 0.9675141694839946]
Number of roots:  26
Rounded off to 6 decimal places:  [0.019267, 0.057745, 0.096425, 0.134613, 0.173729, 0.211335, 0.251182, 0.287914, 0.328784, 0.364348, 0.406541, 0.440633, 0.484458, 0.516762, 0.56255, 0.592724, 0.64084, 0.668493, 0.719368, 0.744029, 0.798218, 0.819249, 0.877575, 0.893966, 0.958104, 0.967514]
Time taken:  0.0023577213287353516  seconds
("Root found by Muller's Method", 0.01926667444973937)
('Root found by Newton Raphson', 0.0577454771

# Time taken if the code is run without any method-tracking

### One can change $K$ in the code to set the number of times the main loop runs

In [47]:
import time
import math
import numpy as np


def f(x):
    return x**2+math.cos(30*math.e*x)

def Df(x):
    return 2*x-30*math.e*math.sin(30*math.e*x)

step=2*math.pi/(2*30*math.e)

def sgn(x):      # The signum function
    if(x>0):
        return 1
    elif(x<0):
        return -1
    else:
        return 0



def newtonraphson(initial_approx):
    x=initial_approx
    if(Df(x)==0):
        return None
    x_new=x-f(x)/Df(x)
    while(abs(f(x_new))>10**(-6) or (abs(x_new-x)/abs(x_new))>10**(-6)):
        x=x_new
        if(Df(x)==0):
            return None
        x_new=x-f(x)/Df(x)
        if(x_new<initial_approx-step/2 or x_new>initial_approx+step/2):
            return None
        if(f(x_new)==0):
            return x_new
    return x_new


def mullermethod(x0,x1,x2,epsilon):
    init_left=x0
    init_right=x2
    init_mid=x1
    epsilon=10**(-6)
    while(abs(x1-x2)/abs(x2)>epsilon or abs(f(x2))>epsilon):
        
        a=((x1-x2)*(f(x0)-f(x2))-(x0-x2)*(f(x1)-f(x2)))/((x0-x2)*(x1-x2)*(x0-x1))    # Coefficients of the quadratic polynomial
        b=((x0-x2)**2*(f(x1)-f(x2))-(x1-x2)**2*(f(x0)-f(x2)))/((x0-x2)*(x1-x2)*(x0-x1)) 
        c=f(x2)
        if(b**2-4*a*c<0): #check if the discriminant is negative, if yes, return None
            return None

        x3=x2-2*c/(b+sgn(b)*math.sqrt(b**2-4*a*c)) # Muller's formula

        x0,x1,x2=x1,x2,x3
        if(x0<init_left or x2>init_right):  # Checking if the approximation is in the interval,                 
            return None     #if not, stop the iteration and return None
    return x3






def secantmethod(init_left,init_right):
    a=init_left
    b=init_right
    if(f(b)==f(a)):
        return None
    c=(a*f(b)-b*f(a))/(f(b)-f(a))
    while(abs(f(c))>10**(-6) or abs(c-b)/abs(c)>10**(-6)):
        a=b
        b=c
        if(f(b)==f(a)):
            return None
        c=(a*f(b)-b*f(a))/(f(b)-f(a))
        if(f(c)==0):
            return c
        if(a<init_left or b>init_right):
            return None
    return c
        


def regulafalsi(init_left,init_right):
    a=init_left
    b=init_right
    if(f(b)==f(a)):
        return None
    c=(a*f(b)-b*f(a))/(f(b)-f(a))
    old=b
    current=c
    while(abs(f(current))>10**(-6) or abs(current-old)/abs(old)>10**(-6)):
        old=c
        if(f(b)==f(a)):
            return None
        c=(a*f(b)-b*f(a))/(f(b)-f(a))
        if(f(c)==0):
            return c
        if(f(a)*f(c)<0):
            b=c
            current=b
        else:
            a=c
            current=a
    return current 
    


def bisection(init_left,init_right):
    a=init_left
    b=init_right
    old=b
    c=(a+b)/2
    current=c
    while(abs(f(current))>10**(-6) or abs(current-old)/abs(current)>10**(-6)):
        old=current
        c=(a+b)/2
        if(f(c)*f(a)<0):
            b=c
            current=b
        else:
            if(f(c)==0):
                return c
            else:
                a=c
                current=a
    return current

fail_count={"Newton Raphson":0,"Muller's Method":0,"Secant Method":0,"Regula Falsi":0}  # Dictionary to keep track of the number of failures in each method

def root_finder(x,y):
    newtonraphson_output=newtonraphson(np.random.uniform(x,x+step))   # Initial approximation for Newton Raphson is a random number in [x,x+step]
                                                                #Change this to (x+step)/2 to use the midpoint as the initial approximation
    if(newtonraphson_output!=None):
        return newtonraphson_output
    fail_count["Newton Raphson"]+=1
    mullermethod_output=mullermethod(x,(x+y)/2,y,10**(-6))   # Initial approximations for Muller's Method are the endpoints and the midpoint
    if(mullermethod_output!=None):
        return mullermethod_output
    fail_count["Muller's Method"]+=1
    secantmethod_output=secantmethod(x,y)
    if(secantmethod_output!=None):
        return secantmethod_output
    fail_count["Secant Method"]+=1
    regulafalsi_output=regulafalsi(x,y)
    if(regulafalsi_output!=None):
        return regulafalsi_output
    fail_count["Regula Falsi"]+=1
    bisection_output=bisection(x,y)
    
    return bisection_output



run_count=0

time_list=list()
K=1000       # Number of times the main loop is run

while(run_count<=K):
    run_count+=1
    start=time.time()
    root_list=list()
    endpoints=[]
    for i in np.arange(0,1,step):
        if(f(i)*f(i+step)<0):
            endpoints+=[i]
        if(f(i)==0):
            root_list+=[i]

    for i in range(len(endpoints)):
        root_list+=[root_finder(endpoints[i],endpoints[i]+step)]

    end=time.time()
    time_list+=[end-start]



print("Time taken by running the main loop",K,"times:",np.mean(time_list)) # Mean time taken by the program to run the loop K times
print("List of roots:",root_list)
print("Number of roots:",len(root_list))
print("Number of failures in methods (Given that the previous has failed):\n",fail_count, "out of total",K*len(root_list),"root finding attempts")


Time taken by running the main loop 1000 times: 0.0008187570295610151
List of roots: [0.019266674449750325, 0.05774547712972399, 0.0964246286585334, 0.1346126392314707, 0.1737292681927322, 0.2113354824153519, 0.25118178520187634, 0.28791416280047183, 0.32878425885198137, 0.36434765115466805, 0.4065406272521109, 0.4406327178252042, 0.484458202421862, 0.5167624229346501, 0.5625501020084771, 0.592723726065781, 0.6408395285480412, 0.6684932685796484, 0.7193683834581519, 0.7440288265725794, 0.7982180285306593, 0.8192486113862161, 0.8775750852356653, 0.8939655213594527, 0.9581044457089049, 0.9675141694839824]
Number of roots: 26
Number of failures in methods (Given that the previous has failed): {'Newton Raphson': 7168, "Muller's Method": 0, 'Secant Method': 0, 'Regula Falsi': 0} out of total 26000 root finding attempts


# If midpoint is taken as initial approx for Newton's method

In [48]:
import time
import math
import numpy as np


def f(x):
    return x**2+math.cos(30*math.e*x)

def Df(x):
    return 2*x-30*math.e*math.sin(30*math.e*x)

step=2*math.pi/(2*30*math.e)


def sgn(x):      # The signum function
    if(x>0):
        return 1
    elif(x<0):
        return -1
    else:
        return 0


def newtonraphson(initial_approx):
    x=initial_approx
    if(Df(x)==0):
        return None
    x_new=x-f(x)/Df(x)
    while(abs(f(x_new))>10**(-6) or (abs(x_new-x)/abs(x_new))>10**(-6)):
        x=x_new
        if(Df(x)==0):
            return None
        x_new=x-f(x)/Df(x)
        if(x_new<initial_approx-step/2 or x_new>initial_approx+step/2):
            return None
        if(f(x_new)==0):
            return x_new
    return x_new


def mullermethod(x0,x1,x2,epsilon):
    init_left=x0
    init_right=x2
    init_mid=x1
    epsilon=10**(-6)
    while(abs(x1-x2)/abs(x2)>epsilon or abs(f(x2))>epsilon):
        
        a=((x1-x2)*(f(x0)-f(x2))-(x0-x2)*(f(x1)-f(x2)))/((x0-x2)*(x1-x2)*(x0-x1))    # Coefficients of the quadratic polynomial
        b=((x0-x2)**2*(f(x1)-f(x2))-(x1-x2)**2*(f(x0)-f(x2)))/((x0-x2)*(x1-x2)*(x0-x1)) 
        c=f(x2)
        if(b**2-4*a*c<0): #check if the discriminant is negative, if yes, return None
            return None

        x3=x2-2*c/(b+sgn(b)*math.sqrt(b**2-4*a*c)) # Muller's formula

        x0,x1,x2=x1,x2,x3
        if(x0<init_left or x2>init_right):  # Checking if the approximation is in the interval,                 
            return None     #if not, stop the iteration and return None
    return x3






def secantmethod(init_left,init_right):
    a=init_left
    b=init_right
    if(f(b)==f(a)):
        return None
    c=(a*f(b)-b*f(a))/(f(b)-f(a))
    while(abs(f(c))>10**(-6) or abs(c-b)/abs(c)>10**(-6)):
        a=b
        b=c
        if(f(b)==f(a)):
            return None
        c=(a*f(b)-b*f(a))/(f(b)-f(a))
        if(f(c)==0):
            return c
        if(a<init_left or b>init_right):
            return None
    return c
        


def regulafalsi(init_left,init_right):
    a=init_left
    b=init_right
    if(f(b)==f(a)):
        return None
    c=(a*f(b)-b*f(a))/(f(b)-f(a))
    old=b
    current=c
    while(abs(f(current))>10**(-6) or abs(current-old)/abs(old)>10**(-6)):
        old=c
        if(f(b)==f(a)):
            return None
        c=(a*f(b)-b*f(a))/(f(b)-f(a))
        if(f(c)==0):
            return c
        if(f(a)*f(c)<0):
            b=c
            current=b
        else:
            a=c
            current=a
    return current 
    


def bisection(init_left,init_right):
    a=init_left
    b=init_right
    old=b
    c=(a+b)/2
    current=c
    while(abs(f(current))>10**(-6) or abs(current-old)/abs(current)>10**(-6)):
        old=current
        c=(a+b)/2
        if(f(c)*f(a)<0):
            b=c
            current=b
        else:
            if(f(c)==0):
                return c
            else:
                a=c
                current=a
    return current

fail_count={"Newton Raphson":0,"Muller's Method":0,"Secant Method":0,"Regula Falsi":0}  # Dictionary to keep track of the number of failures in each method

def root_finder(x,y):
    newtonraphson_output=newtonraphson((x+y)/2)  # Initial approximation for Newton Raphson is the midpoint of the interval
                                                                
    if(newtonraphson_output!=None):
        return newtonraphson_output
    fail_count["Newton Raphson"]+=1
    mullermethod_output=mullermethod(x,(x+y)/2,y,10**(-6))   # Initial approximations for Muller's Method are the endpoints and the midpoint
    if(mullermethod_output!=None):
        return mullermethod_output
    fail_count["Muller's Method"]+=1
    secantmethod_output=secantmethod(x,y)
    if(secantmethod_output!=None):
        return secantmethod_output
    fail_count["Secant Method"]+=1
    regulafalsi_output=regulafalsi(x,y)
    if(regulafalsi_output!=None):
        return regulafalsi_output
    fail_count["Regula Falsi"]+=1
    bisection_output=bisection(x,y)
    
    return bisection_output



run_count=0

time_list=list()
K=1000       # Number of times the main loop is run

while(run_count<=K):
    run_count+=1
    start=time.time()
    root_list=list()
    endpoints=[]
    for i in np.arange(0,1,step):
        if(f(i)*f(i+step)<0):
            endpoints+=[i]
        if(f(i)==0):
            root_list+=[i]

    for i in range(len(endpoints)):
        root_list+=[root_finder(endpoints[i],endpoints[i]+step)]

    end=time.time()
    time_list+=[end-start]



print("Time taken by running the main loop",K,"times:",np.mean(time_list)) # Mean time taken by the program to run the loop K times
print("List of roots:",root_list)
print("Number of roots:",len(root_list))
print("Number of failures in methods (Given that the previous has failed):\n",fail_count, "out of total",K*len(root_list),"root finding attempts")


Time taken by running the main loop 1000 times: 0.00043862063687045375
List of roots: [0.019266674449750325, 0.05774547712180606, 0.09642462865853343, 0.13461263923147085, 0.173729268192728, 0.21133548241541403, 0.2511817852018763, 0.2879141628004701, 0.32878425885198137, 0.36434765115466805, 0.40654062725211176, 0.44063271782520425, 0.48445820242186144, 0.5167624229346517, 0.5625501020084189, 0.5927237260658423, 0.6408395285443326, 0.6684932685707354, 0.7193683834581172, 0.7440288265725794, 0.7982180285305501, 0.8192486113856497, 0.877575085230593, 0.8939655213724633, 0.958104445708378, 0.9675141694854374]
Number of roots: 26
Number of failures in methods (Given that the previous has failed):
 {'Newton Raphson': 0, "Muller's Method": 0, 'Secant Method': 0, 'Regula Falsi': 0} out of total 26000 root finding attempts
